In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/LayoutQT')

Mounted at /content/drive/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 33.3 MB/s 
     |████████████████████████████████| 163 kB 70.0 MB/s 
     |████████████████████████████████| 7.6 MB 56.5 MB/s 


In [ ]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
def process_data_for_bert(data):

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    X_tokenized = tokenizer(list(data['text']), padding=True, truncation=True, max_length=512)
    y = list(data['class'])
    
    dataset = Dataset(X_tokenized, y)

    return dataset

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForMaskedLM
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

def runBert(df_train_dataset, df_test_dataset):
    
    batch_size=8

    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=False )


    X_train, X_val, y_train, y_val = train_test_split(df_train_dataset['text'], df_train_dataset['class'], 
                                                    test_size=0.2, 
                                                    random_state=42)


    train_dataset = pd.DataFrame({'text':X_train, 'class':y_train})
    val_dataset = pd.DataFrame({'text':X_val, 'class':y_val})

    
    
    train_dataset = process_data_for_bert(train_dataset)
    val_dataset = process_data_for_bert(val_dataset)
    test_dataset = process_data_for_bert(df_test_dataset)

    # ----- 2. Fine-tune pretrained model -----#
    # Define Trainer parameters
    def compute_metrics(p):
        
        pred, labels = p
        pred = np.argmax(pred, axis=1)

        accuracy = accuracy_score(y_true=labels, y_pred=pred)
        recall = recall_score(y_true=labels, y_pred=pred)
        precision = precision_score(y_true=labels, y_pred=pred)
        f1 = f1_score(y_true=labels, y_pred=pred)

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

    # Define Trainer
    args = TrainingArguments(
        "./model/layoutqt",
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=100,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='f1'
      # optim="adamw_torch"
    )

    def model_init():
        return model

    trainer = Trainer(
        model_init=model_init,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # New code - wrap collator in a dictionary
    #data_collator = trainer.data_collator
    #trainer.data_collator = lambda data: dict(data_collator(data))
    # End new code

    trainer.train()

    # Make prediction
    raw_pred, _, _ = trainer.predict(test_dataset)

    # Preprocess raw predictions
    y_pred = np.argmax(raw_pred, axis=1)

    return y_pred
    #return roc_auc_score(df_test_dataset['class'], y_pred), f1_score(df_test_dataset['class'], y_pred, average='macro')

In [ ]:
import numpy as np
import pandas as pd

             
df_train = pd.read_csv("./input/tobacco800/df_train.csv")
df_test = pd.read_csv("./input/tobacco800/df_test.csv")

df_train['class'] = df_train['class'].apply(lambda x: 1 if x=="FirstPage" else 0)
df_test['class'] = df_test['class'].apply(lambda x: 1 if x=="FirstPage" else 0)

df_train['text'] = df_train['text'].apply(lambda x: str(x))
df_test['text'] = df_test['text'].apply(lambda x: str(x))

y_pred = runBert(df_train, df_test)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 824
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10300


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.405177,0.835749,0.820312,0.905172,0.860656
2,No log,0.335200,0.898551,0.913043,0.905172,0.909091
3,No log,0.397784,0.888889,0.872000,0.939655,0.904564
4,No log,0.386799,0.903382,0.893443,0.939655,0.915966
5,0.327700,0.479933,0.879227,0.864000,0.931034,0.896266
6,0.327700,0.590824,0.879227,0.876033,0.913793,0.894515
7,0.327700,0.687164,0.874396,0.881356,0.896552,0.888889
8,0.327700,0.574801,0.879227,0.876033,0.913793,0.894515
9,0.327700,0.883987,0.864734,0.879310,0.879310,0.879310
10,0.097700,0.935717,0.864734,0.843750,0.931034,0.885246


***** Running Evaluation *****
  Num examples = 207
  Batch size = 8
Saving model checkpoint to ./model/layoutqt/checkpoint-103
Configuration saved in ./model/layoutqt/checkpoint-103/config.json
Model weights saved in ./model/layoutqt/checkpoint-103/pytorch_model.bin
tokenizer config file saved in ./model/layoutqt/checkpoint-103/tokenizer_config.json
Special tokens file saved in ./model/layoutqt/checkpoint-103/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 207
  Batch size = 8
Saving model checkpoint to ./model/layoutqt/checkpoint-206
Configuration saved in ./model/layoutqt/checkpoint-206/config.json
Model weights saved in ./model/layoutqt/checkpoint-206/pytorch_model.bin
tokenizer config file saved in ./model/layoutqt/checkpoint-206/tokenizer_config.json
Special tokens file saved in ./model/layoutqt/checkpoint-206/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 207
  Batch size = 8
Saving model checkpoint to ./model/layoutqt/checkpoint-

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

print("Accuracy:", accuracy_score(df_test['class'], y_pred))
print("F1:", f1_score(df_test['class'], y_pred, average='macro'))
print("ROC_AUC:", roc_auc_score(df_test['class'], y_pred))


Accuracy: 0.9227799227799228
F1: 0.9207951070336391
ROC_AUC: 0.9207951070336392


In [ ]:
print(classification_report(df_test['class'], y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91       109
           1       0.93      0.93      0.93       150

    accuracy                           0.92       259
   macro avg       0.92      0.92      0.92       259
weighted avg       0.92      0.92      0.92       259

